<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BLOOM/blob/main/ExemplosGeracaoTexto_BLOOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando BLOOM Transformers by HuggingFace

https://huggingface.co/docs/transformers/model_doc/bloom

https://huggingface.co/blog/bloom-megatron-deepspeed

https://huggingface.co/blog/zero-shot-eval-on-the-hub


# **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplo de geração de textos em:
- https://huggingface.co/spaces/huggingface/bloom_demo
- https://studyexperts.in/blog/how-to-use-the-bloom-model-using-python/


Treinamento:
- Hardware	384 80GB A100 GPUs
- Software	Megatron-DeepSpeed
- Arquitetura GPT3 w/ extras
- Conjunto de dados de 350 Bilhões de tokens de 59 linguas
- Tempo de treinamento	3.5 meses

**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

# 1 - Instalação Transformer da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.23.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 48.0 MB/s 
     |████████████████████████████████| 7.6 MB 35.9 MB/s 


# 2 - Carregando o BLOOM

O modelo BLOOM foi proposto com suas várias versões através do BigScience Workshop. A BigScience é inspirada em outras iniciativas de ciência aberta onde os pesquisadores reuniram seu tempo e recursos para alcançar coletivamente um impacto maior. A arquitetura do BLOOM é essencialmente semelhante ao GPT3 (modelo auto-regressivo para previsão do próximo token), mas foi treinado em 46 linguagens diferentes e 13 linguagens de programação. Várias versões menores dos modelos foram treinadas no mesmo conjunto de dados. BLOOM está disponível nas seguintes versões:


Modelos:
 - bigscience-small-testing (32,3 MB) - Ok Colab
 - bloom-560m (1,12 GB) - Ok Colab 
 - bloom-1b1 (2,13 GB) - Ok Colab
 - bloom-1b7 (3,44 GB) - Não carrega no Colab
 - bloom-3b (6,01 GB) - Não carrega no Colab
 - bloom-7b1 (9,98 + 4,16 GB) - Não carrega no Colab
 - bloom (176 Bilhões de parâmetros) (7,19 GB + 44 * 4,93GB) = 224,11 GB - Não carrega no Colab


Lista dos modelos:
  - https://huggingface.co/models?other=bloom

In [ ]:
# modelo_bloom = "bigscience-small-testing"
# modelo_bloom = "bigscience/bloom-560m"
modelo_bloom = "bigscience/bloom-1b1"

# modelo_bloom = "bigscience/bloom-1b7"
# modelo_bloom = "bigscience/bloom-3b"
# modelo_bloom = "bigscience/bloom-7b1"
# modelo_bloom = "bigscience/bloom"

## 2.1 Carrega o tokenizador

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modelo_bloom)

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

## 2.2 - Carregando o Modelo BLOOM(AutoModelForCausalLM)



In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(modelo_bloom, use_cache=True)

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

# 3 - Exemplos de geração de textos com BLOOM

## 3.1 - Exemplo geração de texto


In [ ]:
# Define o documento base
# documento = "Como empilhar elementos em muma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in zip(input['input_ids'][0], input['attention_mask'][0]):
    # print(tup[0], tup[1])
    print("{:>3} {} {} {}".format(i, tokenizer.convert_ids_to_tokens(tup[0].item()), tup[0], tup[1]))
    i= i + 1

  0 Bom 82152 1
  1 Ġdia 2515 1
  2 Ġprofessor 26804 1
  3 , 15 1
  4 Ġtudo 11845 1
  5 Ġbem 7658 1
  6 Ġ? 2040 1


In [ ]:
# Ele obterá o resultado e podemos fornecer o número máximo de palavras como parâmetro
# podemos reduzir o plágio ajustando o valor da temperatura mais próximo de 1

exemplo = model.generate(**input, max_length=100, top_k=0, temperature=0.7)

In [ ]:
# Mostra o resultado
print(tokenizer.decode(exemplo[0]))

# print(tokenizer.decode(exemplo.squeeze(), skip_special_tokens=True))

Bom dia professor, tudo bem?Estou com uma dúvida, eu fiz um projeto de um sistema de controle de acesso, e o sistema de controle de acesso é feito com o sistema de controle de acesso de um computador, e o sistema de controle de acesso é feito com o sistema de controle de acesso de um computador, e o sistema de controle de acesso é feito com o sistema de controle de acesso de um computador, e o sistema de controle de acesso é feito com o sistema de controle de acesso


## 3.2 - Outro exemplo

https://pythontechworld.com/issue/huggingface/transformers/18809

In [ ]:
import torch

def compute_gen_loss(lm_logits: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    batch_size = labels.shape[0]
    shift_logits = lm_logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()

    loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1)
    )
    loss = loss.reshape(batch_size, -1)
    loss = loss.sum(dim=-1) / (shift_labels != -100).sum(dim=-1)
    
    return loss

In [ ]:
def pad_ids(arrays, padding, max_length=-1):
    if (max_length < 0):
        max_length = max(list(map(len, arrays)))

    arrays = [[padding] * (max_length - len(array)) + array for array in arrays]

    return arrays

In [ ]:
def forward(text: list, labels: str, conditional: bool = True):

    input_tokens = tokenizer(text).input_ids
    label_tokens = tokenizer(labels).input_ids

    input_ids = [x + y for (x, y) in zip(input_tokens, label_tokens)]
    attention_mask = [(len(x) + len(y)) * [1] for (x, y) in zip(input_tokens, label_tokens)]
    if (conditional):
        labels = [[-100] * len(x) + y for (x, y) in zip(input_tokens, label_tokens)]
    else:
        labels = input_ids

    pad = 3
    input_ids = pad_ids(input_ids, pad)
    attention_mask = pad_ids(attention_mask, 0)    
    # rótulos precisam estar no dispositivo de saída
    labels = pad_ids(labels, -100)

    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    labels = torch.tensor(labels)
    
    # submete ao modelo
    lm_logits = model(
        input_ids=input_ids,
        attention_mask=attention_mask
    ).logits

    # Mostra o resultado
    print(compute_gen_loss(lm_logits, labels).cpu().tolist())


In [ ]:
text = [
    # "DeepSpeed",
    # "DeepSpeed is a",
    # "DeepSpeed is a machine",
    # "DeepSpeed is a machine learning framework",
    "Deep Speed",
    "DeepSpeed é um",
    "DeepSpeed é uma máquina",
    "DeepSpeed é uma estrutura de aprendizado de máquina",
]
labels = [
    # " is awesome.",
    # " good person.",
    # " that can wipe out the planet.",
    # " for generating memes.",
    " é incrível.",
    " boa pessoa.",
    " que pode acabar com o planeta.",
    " para gerar memes.",
]

print("Original")
forward(text, labels)

# labels[0] = " is awesome. really awesome"
labels[0] = " é incrível. realmente incrível"
forward(text, labels)

#labels[0] = " is awesome. really awesome. Try it."
labels[0] = " é incrível. realmente incrível. Experimente."
forward(text, labels)

# labels[0] = " is awesome. really awesome. Try it. You'll be surprised"
labels[0] = " é incrível. realmente incrível. Experimente. Você ficará surpreso"
forward(text, labels)

# labels[0] = " is awesome. really awesome. Try it. You'll be surprised. BLOOM was trained using DeepSpeed."
labels[0] = " é incrível. realmente incrível. Experimente. Você ficará surpreso. BLOOM foi treinado usando DeepSpeed."
forward(text, labels)

# labels[0] = " is awesome. really awesome. Try it. You'll be surprised. BLOOM was trained using DeepSpeed. Oh no the values are bugging out now."
labels[0] = " é incrível. realmente incrível. Experimente. Você ficará surpreso. O BLOOM foi treinado usando DeepSpeed. Ah, não, os valores estão bugados agora."
forward(text, labels)

Original
[5.990021228790283, 7.568333148956299, 3.1083452701568604, 4.449141979217529]
[6.825961112976074, 7.568333148956299, 3.1083452701568604, 4.449141979217529]
[5.244286060333252, 7.568333148956299, 3.1083452701568604, 4.449141979217529]
[4.190319538116455, 7.568333148956299, 3.108344793319702, 4.449139595031738]
[4.509304523468018, 7.568332672119141, 3.108344554901123, 4.449138641357422]
[4.370351791381836, 7.568332195281982, 3.1083462238311768, 4.4491400718688965]
